# Introduction to Brain Segmentation with Keras

## MAIN 2019 Educational Course 

### Thomas Funck

### McGill University

### **Contact**: email: [tffunck@gmail.com](mailto:tffunck@gmail.com) , Twitter: [@tffunck\](https://twitter.com/tffunck)

## Configuring basic options

In [31]:
from minc_keras import create_dir_verbose
from utils import *

### Set input and label string
input_str='pet.mnc' 
label_str='dtissue.mnc'

### Set filename for .csv that will store data frame 
images_fn='data.csv'

### Set source directory from which data will be read
source_dir="/data1/users/tfunck/pet/data_ses/"

### Set the target directory where output results will be saved
target_dir="/data1/users/tfunck/pet/results"

### Set raiots for train/validation/test
ratios=[0.7,0.15]

### By default we set clobber to False so that we don't overwrite existing files
### Feel free to change if needed
clobber=True

### Size of batches that will be passed to model. The default 2 makes it easy
batch_size=2

### Image dimensions. We are slicing the 3D images into 2D slices. This serves to augment the data
### and make training faster
image_dim=2

### Output activation function
activation_output="softmax"

## Create output directories
### This is just a little housekeeping

In [32]:
### Create output directories to save results
data_dir = target_dir + os.sep + 'data'+os.sep
report_dir = target_dir+os.sep+'report'+os.sep
train_dir = target_dir+os.sep+'predict'+os.sep+'train'+os.sep
test_dir = target_dir+os.sep+'predict'+os.sep+'test'+os.sep
validate_dir = target_dir+os.sep+'predict'+os.sep+'validate'+os.sep
model_dir=target_dir+os.sep+'model'

create_dir_verbose(train_dir)
create_dir_verbose(test_dir)
create_dir_verbose(validate_dir)
create_dir_verbose(data_dir)
create_dir_verbose(report_dir) 
create_dir_verbose(model_dir)   

### Set filename for .csv file that will contain info about input images
images_fn = set_model_name(images_fn, report_dir, '.csv')

## Organize input and label images into train/validate/test splits
#### One of the hard parts of doing deep learning in practice is organizing your data.
#### The following section is a bit of complicated because it involves organizing the input and label files into 
#### a data frame and assigning them to a train/validate/test splits.
#### ***You can skip this if you're primarily interested in how to build a network.***

In [ ]:
    from prepare_data import *
    from set_images import *
    ext='mnc' #default file extension is mnc for MINC files
    ################
    # Prepare Data #
    ################
    data={}
    ### 0) Setup file names and output directories
    data["train_x_fn"] = data_dir + os.sep + 'train_x'
    data["train_y_fn"] = data_dir + os.sep + 'train_y'
    data["validate_x_fn"] = data_dir + os.sep + 'validate_x'
    data["validate_y_fn"] = data_dir + os.sep + 'validate_y'
    data["test_x_fn"] = data_dir + os.sep + 'test_x'
    data["test_y_fn"] = data_dir + os.sep + 'test_y'
    
    ### 1) Organize inputs into a data frame, match each PET image with label image
    if not exists(images_fn) or clobber:
        ### set_images is a very important function that will find all the PET images and their
        ### corresponding labelled images from source_dir. This function uses <input_str> and <label_str>
        ### to identify which files are inputs and labeles, respectively. The images use the BIDS file format
        ### where subject, session, task, radiotracer are specificied in the filename. These variables are parsed
        ### from the filenames and also stored in the data frame. 
        ### set_images will also split the images into training, validation, and test subsets
        
        ##############
        # Set Images #
        ##############
        # 1 - gathering information (parsing the source directory)
        subject_dirs, pet_list, names = gather_dirs(source_dir, input_str, ext )
    
        # 2 - checking for potential errors
        if len(names) == 0:
            print_error_nosubject(source_dir)
        if sum(ratios) > 1.:
            print_error_nosumone(ratios)
    
        # 3 - creating an empty directory of dataframes, then filling it.
        dfd = {}
        for name in names:
            data_subject = process(name, source_dir, pet_list, label_str, ext)
            #data_subject = process(name, source_dir, pet_list, t1_list, label_str, ext)
            if not data_subject == 1: dfd[name] = pd.DataFrame(data_subject)  # formerly subject_df
            
        # 4 - concatenation of the dict of df to a single df
        out = create_out(dfd)
        
        # 5 - attributing a train/validate/test category for all subject
        # By setting the <category_class> to "radiotracer", the function <attribute_category> will attempt to split
        # the amount of subjects for each radiotracer evenly into train/validate/test splits
        if "radiotracer" in out.columns : category_class="radiotracer" 
        else : category_class="task" 
            
        ### <attribute_category> will divide the subjects into train/validate/test splits, making sure that 
        ### if there are multiple scans from the same subject that they are all in the same split. For example,
        ### you don't want the FDOPA scan for subject 01 to be in the train split and the FDG scan for subject
        ### 01 in the validate split.
        attribute_category(out, 'train',category_class, ratios[0])
        attribute_category(out, 'validate',category_class, ratios[1])
        out.category.loc[ out.category=="unknown" ] = "test"
            
        #5.5 Set the number of valid samples per image (some samples exluded because they contain no information)
        set_valid_samples(out)

        # 6 - export and return
        out.to_csv(images_fn, index=False)
        
    else: 
        images = pd.read_csv(images_fn)
            
    ### 2) Split images into training and validate data frames
    ###
    train_images = images[images['category']=='train'].reset_index()
    validate_images = images[images['category']=='validate'].reset_index()
    test_images = images[images['category']=='test'].reset_index()
    train_valid_samples = train_images.valid_samples.values.sum()  
    validate_valid_samples  =  validate_images.valid_samples.values.sum()

    ### 3) Get spatial dimensions of images 
    data["image_dim"] = get_image_dim(images.iloc[0].label)

    ### 4) Set up dimensions of data tensors to be used for training and validateing. all of the
    if not exists(data["train_x_fn"] + '.npy') or not exists( data["train_y_fn"] + '.npy') or clobber:
        feature_extraction(train_images, data['image_dim'], data["train_x_fn"], data["train_y_fn"], data_dir, clobber)
    if not exists(data["validate_x_fn"] + '.npy') or not exists(data["validate_y_fn"] + '.npy') or clobber:
        feature_extraction(validate_images, data['image_dim'], data["validate_x_fn"], data["validate_y_fn"], data_dir, clobber)
    if not exists(data["test_x_fn"] + '.npy') or not exists(data["test_y_fn"] + '.npy') or clobber:
        feature_extraction(validate_images, data['image_dim'], prepare_data["test_x_fn"], data["test_y_fn"], data_dir, clobber)
    data["batch_size"] = adjust_batch_size(train_valid_samples, validate_valid_samples, batch_size)

train : expected/real ratio = 70.00 / 70.29
validate : expected/real ratio = 15.00 / 15.22


## Building a U-NET in Keras

![](https://github.com/tfunck/minc_keras/blob/master/images/unet.png?raw=1)


In [ ]:
    ### 1) Define architecture of neural network
    Y_validate=np.load(data["validate_y_fn"]+'.npy')
    nlabels=len(np.unique(Y_validate))#Number of unique labels in the labeled images
    
    img_rows=image_dim[1]
    img_cols=image_dim[2]
    nMLP=16
    nRshp=int(sqrt(nMLP))
    nUpSm=int(image_dim[0]/nRshp)
    image = Input(shape=(image_dim[1], image_dim[2],1))
    
    BN1 = BatchNormalization()(image)

In [ ]:
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

In [ ]:
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

In [ ]:
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

In [ ]:
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

In [ ]:
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

In [ ]:
    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

In [ ]:
    conv6_up = UpSampling2D(size=(2, 2))(conv6)
    conv6_pad = ZeroPadding2D( ((1,0),(1,0)) )(conv6_up)
    up7 = merge([conv6_pad, conv3], mode='concat', concat_axis=3)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

In [ ]:
    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

In [ ]:
    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

In [ ]:
    conv10 = Convolution2D(nlabels, 1, 1, activation=activation)(conv9)

    model = keras.models.Model(input=[image], output=conv10)

In [ ]:
### 2) Train network on data
model_fn =set_model_name(model_fn, model_dir)
history_fn = splitext(model_fn)[0] + '_history.json'

print( 'Model:', model_fn)
if not exists(model_fn) or clobber:
#If model_fn does not exist, or user wishes to write over (clobber) existing model
#then train a new model and save it
    X_train=np.load(data["train_x_fn"]+'.npy')
    Y_train=np.load(data["train_y_fn"]+'.npy')
    X_validate=np.load(data["validate_x_fn"]+'.npy')
     model,history = compile_and_run(model, model_fn, history_fn, X_train,  Y_train, X_validate,  Y_validate, nb_epoch, nlabels, loss=loss)


In [ ]:
    ### 3) Evaluate model on test data
    model = load_model(model_fn)
    X_test=np.load(data["test_x_fn"]+'.npy')
    Y_test=np.load(data["test_y_fn"]+'.npy')
    if loss in categorical_functions :
        Y_test=to_categorical(Y_test)
    test_score = model.evaluate(X_test, Y_test, verbose=1)
    print('Test: Loss=', test_score[0], 'Metric=', test_score[1])
    #np.savetxt(report_dir+os.sep+'model_evaluate.csv', np.array(test_score) )

    ### 4) Produce prediction
    #predict(model_fn, validate_dir, data_dir, images_fn, images_to_predict=images_to_predict, category="validate", verbose=verbose)
    #predict(model_fn, train_dir, data_dir, images_fn, images_to_predict=images_to_predict, category="train", verbose=verbose)
    predict(model_fn, test_dir, data_dir, images_fn, loss, images_to_predict=images_to_predict, category="test", verbose=verbose)
    plot_loss(metric, history_fn, model_fn, report_dir)